## Setting up a SWAT Connection for Python

In [ ]:
import os
import swat

In [ ]:
os.environ['CAS_CLIENT_SSL_CA_LIST'] = R"C:\SAS Viya\CAS_ex_cert.crt"

In [ ]:
conn = swat.CAS(
    hostname="sas-cas-server-default-bin-demo.uksouth.cloudapp.azure.com",
    port=5570,
    authinfo=R"C:\SAS Viya\authinfo"
)

In [ ]:
conn.serverstatus()

## Create a reference to in-memory tables

In [ ]:
gyms = conn.CASTable("gym_summary", caslib="public")
members = conn.CASTable("members", caslib="public")

## Use pandas code and functions on the CAS tables as if they were standard dataframes

These will be translated into CAS actions, and alter the underlying table in CAS

In [ ]:
# Get head data from table, methods similar to pandas
gyms.head()

In [ ]:
# Work with the tables as if they were pandas dataframes

gyms["achieving_target"] = gyms["members_net_change"] > 0

In [ ]:
members_subset = members[members.country == "England"]

members_subset.head()

## Use the full capabilities of CAS

Can load in Action Sets to perform any CAS action, i.e. powerful in-memory & distributed analysis

In [ ]:
conn.loadactionset("regression")

In [ ]:
ouput = gyms.glm(target = "total_revenue",
         inputs = ["achieving_target", "Gym_Size", "Gym_Current_Members"],
         output = {
             "casOut": {"name": "RevenuePrediction", "replace": True},
             "copyvars": "all",
             "pred": "Predicted_Revenue",
         }
)

In [ ]:
result = conn.CASTable("RevenuePrediction")

In [ ]:
result.head()

In [ ]:
result.plot.scatter("Predicted_Revenue", "total_revenue")